In [61]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import SGDRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from scipy.stats import t

In [62]:
df=pd.read_csv(r"./preprocessed.csv",index_col=0)
df

,hasImage,has3DModel,hasAdditionalAttributions,marketingStatusSimplifiedCd_Pre-Foreclosure,marketingStatusSimplifiedCd_RecentChange,statusText_Sold,sgapt_Unknown Listed By,city_Charlotte,city_Chicago,city_Denver,...,homeType_LOT,homeType_MANUFACTURED,homeType_MULTI_FAMILY,homeType_SINGLE_FAMILY,homeType_TOWNHOUSE,beds,baths,area,rentZestimate,price
0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,3.0,2.0,1224.0,3999.0,819500.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,723.0,3634.0,880000.0
2,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,6.0,3.0,3396.0,6000.0,1250000.0
3,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,4.0,3.0,2400.0,4964.0,805000.0
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,9.0,3.0,4107.0,8970.0,650000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6201,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,5.0,4.0,2366.0,4999.0,1125000.0
6202,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1802.0,3449.0,465000.0
6203,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,4.0,4.0,2875.0,4749.0,1020000.0
6204,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,4.0,2.5,1260.0,3994.0,550000.0


In [63]:
def remove_outliers(X,y,cols=[], threshold=3):
    z_scores = (X[cols] - X[cols].mean()) / X[cols].std()
    index=(np.abs(z_scores)<threshold).all(axis=1)
    return X[index],y[index]

def scale_selected_columns(x_train, x_test, cols):
    scaler = MinMaxScaler()
    x_train_scaled = x_train.copy()  
    x_test_scaled = x_test.copy()    
    scaler.fit(x_train_scaled[cols])  
    x_train_scaled[cols] = scaler.transform(x_train_scaled[cols])  
    x_test_scaled[cols] = scaler.transform(x_test_scaled[cols])
    return x_train_scaled, x_test_scaled


In [64]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

def perform_decision_tree_regression_zsor(df):
    r2_list = []
    best_models = []

    param_grid = {
        'max_depth': [None, 3, 5, 10,15,30],
    }

    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train, y_train = remove_outliers(x_train, y_train, ["beds", "baths", "rentZestimate", "area"])

        model = DecisionTreeRegressor(random_state=42)

        grid_search = GridSearchCV(model, param_grid, cv=None, scoring='r2')
        grid_search.fit(x_train, y_train)

        best_model = grid_search.best_estimator_
        best_models.append(best_model)

        train_predictions = best_model.predict(x_train)
        test_predictions = best_model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2": train_r2, "test_r2": test_r2})

    return r2_list, best_models

results, best_models = perform_decision_tree_regression_zsor(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)


Mean train_r2: 0.779712691624239
Mean test_r2: 0.5578960993775389


In [65]:

def perform_linear_regression_zsor(df):
    r2_list=[]
    
    for i in range(100):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train,y_train=remove_outliers(x_train,y_train,["beds","baths","rentZestimate","area"])

        model = LinearRegression()
        model.fit(x_train, y_train)

        train_predictions = model.predict(x_train)
        test_predictions = model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2":train_r2,"test_r2":test_r2})

    return r2_list,model

    

results,model=perform_linear_regression_zsor(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)
results


Mean train_r2: 0.7759773703986008
Mean test_r2: 0.8014381434684539


[{'train_r2': 0.7753797540234137, 'test_r2': 0.8087435418511384},
 {'train_r2': 0.7752521770080076, 'test_r2': 0.8153100235421703},
 {'train_r2': 0.7796733631322396, 'test_r2': 0.7854181087621367},
 {'train_r2': 0.7759183668892913, 'test_r2': 0.7872595077357105},
 {'train_r2': 0.7757856084137978, 'test_r2': 0.583457755784353},
 {'train_r2': 0.7778838418784515, 'test_r2': 0.8397982702436814},
 {'train_r2': 0.792272337835895, 'test_r2': 0.7454024009273823},
 {'train_r2': 0.782592704960802, 'test_r2': 0.7826146380527266},
 {'train_r2': 0.7784279032076962, 'test_r2': 0.7825608093534836},
 {'train_r2': 0.773664525750487, 'test_r2': 0.8258064851753775},
 {'train_r2': 0.7717645306593663, 'test_r2': 0.8423452769773196},
 {'train_r2': 0.7856444936829927, 'test_r2': 0.8347579281204691},
 {'train_r2': 0.7695414527850511, 'test_r2': 0.8864418076411102},
 {'train_r2': 0.773748893604929, 'test_r2': 0.7910203352587012},
 {'train_r2': 0.7832850315622552, 'test_r2': 0.8521986691812733},
 {'train_r2': 0

In [66]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

def perform_knn_regression_zsor(df):
    r2_list = []
    best_models = []

    param_grid = {
    'n_neighbors': [n for n in range(1,100,15)],  
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
    }


    for i in range(10):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train, y_train = remove_outliers(x_train, y_train, ["beds", "baths", "rentZestimate", "area"])

        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

        model = KNeighborsRegressor()

        grid_search = GridSearchCV(model, param_grid, cv=None, scoring='r2')
        grid_search.fit(x_train, y_train)

        best_model = grid_search.best_estimator_
        best_models.append(best_model)

        train_predictions = best_model.predict(x_train)
        test_predictions = best_model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)

        r2_list.append({"train_r2": train_r2, "test_r2": test_r2})

    return r2_list, best_models

results, best_models = perform_knn_regression_zsor(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)


Mean train_r2: 0.9999655233427284
Mean test_r2: 0.5038867594910794


In [73]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

def perform_random_forest_regression_zsor(df):
    r2_list = []
    best_models = []

    param_grid = {
        'n_estimators': [100],
        'max_depth': [1,3,5,7,9,11,13,15],
    }

    for i in range(3):
        x = df.drop(columns=["price"])
        y = df["price"]
        
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)

        x_train, y_train = remove_outliers(x_train, y_train, ["beds", "baths", "rentZestimate", "area"])

        scaler = MinMaxScaler()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

        model = RandomForestRegressor()

        grid_search = GridSearchCV(model, param_grid, cv=None, scoring='r2')
        grid_search.fit(x_train, y_train)

        best_model = grid_search.best_estimator_
        best_models.append(best_model)

        train_predictions = best_model.predict(x_train)
        test_predictions = best_model.predict(x_test)

        train_r2 = r2_score(y_train, train_predictions)
        test_r2 = r2_score(y_test, test_predictions)
        print(test_r2)

        r2_list.append({"train_r2": train_r2, "test_r2": test_r2})

    return r2_list, best_models

results, best_models = perform_random_forest_regression_zsor(df)
results

mean_train_r2 = sum(entry['train_r2'] for entry in results) / len(results)
mean_test_r2 = sum(entry['test_r2'] for entry in results) / len(results)

print("Mean train_r2:", mean_train_r2)
print("Mean test_r2:", mean_test_r2)


0.7385739298205558
0.5779385973907873
0.6691111346585984
Mean train_r2: 0.9445802055505305
Mean test_r2: 0.6618745539566472
